In [18]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [2]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.30.130
pinled       2
pindetector  13
boardname    esp8266C

Sent 6 lines (128 bytes) to config.txt.


In [19]:
%sendtofile main.py

import time, network, machine, ubinascii

fconfig = dict(x.split()  for x in open("config.txt"))
pled = machine.Pin(int(fconfig["pinled"]), machine.Pin.OUT)
pdetector = machine.Pin(int(fconfig["pindetector"]), machine.Pin.IN)

si = network.WLAN()
si.active(True)
si.connect(fconfig["wifiname"], fconfig["wifipassword"])
while not si.isconnected():
    time.sleep_ms(500)
ipnumber = si.ifconfig()[0]
macaddress = ubinascii.hexlify(si.config('mac'),':').decode().upper()
print(ipnumber)
for i in range(10):
    pled.value(i%2)
    time.sleep_ms(100)

from umqtt.robust import MQTTClient
client = MQTTClient(fconfig["boardname"], fconfig["mqttbroker"])
while True:
    try:
        client.connect()
        break
    except OSError:
        time.sleep_ms(1000)
for i in range(10):
    pled.value(i%2)
    time.sleep_ms(50)

topicname = fconfig["boardname"]+"/ticks"
topicip = fconfig["boardname"]+"/ip"
topicmac = fconfig["boardname"]+"/mac"

n = 0
t0 = 0
tmacnext0 = 0
while True:
    while pdetector.value():       pass
    td = time.ticks_ms()
    while pdetector.value() == 0:  pass
    t = time.ticks_ms()
    x = (t - td)  # originally machine.time_pulse_us(pdetector, 0, 10000000)//1000
    dt = t - t0
    print(topicname, dt, x, n)
    if not (20 < x < 35):
        continue
    if dt < 15:
        continue
    try:
        client.publish(topicname, "%d %d %d" % (dt, x, n))
        if t > tmacnext0:
            client.publish(topicip, ipnumber)
            client.publish(topicmac, macaddress)
            tmacnext0 = t + 60000
    except Exception as e:
        print(e)
    t0 = t
    n += 1
    pled.value(n%2) 


Sent 61 lines (1618 bytes) to main.py.


In [16]:
client = MQTTClient(fconfig["boardname"], "mqtt.local")



In [17]:
print(client.connect())

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "umqtt/simple.py", line 57, in connect
OSError: -2


In [11]:
client.publish(topicmac, macaddress)


In [ ]:
# esp32 test flashing light signal
%sendtofile main.py
import machine, time
pled = machine.Pin(2, machine.Pin.OUT)
pwm = machine.PWM(pled, freq=1, duty=25)
print(pwm)
